In [35]:
import cv2, os

data_path = "/Volumes/Samsung_T5/Data_Sets/dataset"
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]
label_dict = dict(zip(categories, labels))

print(categories)
print(labels)
print(label_dict)

['with_mask', 'without_mask']
[0, 1]
{'with_mask': 0, 'without_mask': 1}


In [36]:
def get_image_path(folder_path):
#     img = os.listdir('/Volumes/Samsung_T5/Data_Sets/dataset/without_mask')
    path = []
    img = os.listdir(folder_path)
    for i in img:
        img_loc = i.split("._")
        if len(img_loc) > 1:
            pass
        else:
            path.append(img_loc[len(img_loc) - 1])
    return path


In [37]:
img_size = 100
data = []
target = []

for category in categories:
    folder_path = os.path.join(data_path, category)
    img_names = get_image_path(folder_path)
    print(folder_path)
    print(len(img_names))
    
    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
#         print(img_path)
        img = cv2.imread(img_path)
        
        try:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (100, 100))
            data.append(resized)
            target.append(label_dict[category])
        except Exception as e:
            print(f'Exception: {e}')
            
print(f'Target length: {len(target)}')
print(f'Data Lenght: {len(data)}')

/Volumes/Samsung_T5/Data_Sets/dataset/with_mask
690
/Volumes/Samsung_T5/Data_Sets/dataset/without_mask
686
Target length: 1376
Data Lenght: 1376


In [38]:
import numpy as np
from keras.utils import to_categorical

data = np.array(data)/255.0
data = np.reshape(data, (data.shape[0], 100, 100, 1))

target = np.array(target)
new_target = to_categorical(target)

In [5]:
new_target.shape

(1376, 2)

In [67]:
np.save('data', data)
np.save('target', new_target)

In [39]:
import numpy as np

data = np.load('data.npy')
target = np.load('target.npy')

Model for mask detection (CNN)

In [7]:
from keras.models import Sequential
from keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from keras.callbacks import ModelCheckpoint

model = Sequential()

model.add(Conv2D(200, (3, 3), activation = "relu", input_shape = data.shape[1:]))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(100, (3 ,3), activation = "relu", ))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())
model.add(Dropout(0.5))

model.add(Dense(50, activation = "relu"))
model.add(Dense(2, activation = "softmax"))

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ['accuracy'])

In [70]:
target.shape

(1376, 2)

In [42]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_target, test_target = train_test_split(data, target, test_size = 0.1)

In [72]:
model.fit(train_data, train_target, epochs = 20, validation_split = 0.2)

Epoch 1/20
31/31 [==============================] - 35s 1s/step - loss: 0.7499 - accuracy: 0.5707 - val_loss: 0.6719 - val_accuracy: 0.5242
Epoch 2/20
31/31 [==============================] - 34s 1s/step - loss: 0.6292 - accuracy: 0.6303 - val_loss: 0.5156 - val_accuracy: 0.8024
Epoch 3/20
31/31 [==============================] - 33s 1s/step - loss: 0.5094 - accuracy: 0.7616 - val_loss: 0.3956 - val_accuracy: 0.8468
Epoch 4/20
31/31 [==============================] - 33s 1s/step - loss: 0.3638 - accuracy: 0.8545 - val_loss: 0.2847 - val_accuracy: 0.8911
Epoch 5/20
31/31 [==============================] - 33s 1s/step - loss: 0.2409 - accuracy: 0.9232 - val_loss: 0.2367 - val_accuracy: 0.9355
Epoch 6/20
31/31 [==============================] - 33s 1s/step - loss: 0.1958 - accuracy: 0.9263 - val_loss: 0.2329 - val_accuracy: 0.9032
Epoch 7/20
31/31 [==============================] - 33s 1s/step - loss: 0.1513 - accuracy: 0.9535 - val_loss: 0.1768 - val_accuracy: 0.9395
Epoch 8/20
31/31 [==

In [73]:
model.save('mask_detector')

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: mask_detector/assets


Loading the saved Model and evaluating it. 

In [40]:
from tensorflow import keras
model = keras.models.load_model('mask_detector')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 98, 98, 200)       2000      
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 49, 49, 200)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 47, 47, 100)       180100    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 23, 23, 100)       0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 52900)             0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 52900)             0         
_________________________________________________________________
dense_4 (Dense)              (None, 50)               

In [43]:
print(model.evaluate(test_data, test_target))

5/5 [==============================] - 1s 197ms/step - loss: 0.0467 - accuracy: 0.9855
[0.04666541889309883, 0.9855072498321533]


In [44]:
from keras.models import load_model
import numpy as np

face_cascader = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

labels_dict = {0: "WITH MASK", 1: "WITHOUT MASK"}
colors_dict = {0: (0, 255, 0), 1: (0, 0, 255)}

In [34]:
cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture("/Users/shubhamkumar/Documents/Videos/stock_video_files/[4K] Young Man Putting on a Face Mask - FREE STOCK FOOTAGE - NO COPYRIGHT - FOOTAGE 4K.mp4")
# cap = cv2.VideoCapture("/Users/shubhamkumar/Documents/Videos/IMG_3032.MOV")

while(True):
    _, img = cap.read()
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascader.detectMultiScale(gray_image)

    for (x, y, w, h) in faces:
        face_img = gray_image[y: y+w, x: x+w]
        resized = cv2.resize(face_img, (100, 100))
        normalized = resized/255
        reshaped = np.reshape(normalized, (1, 100, 100, 1))
        result = model.predict(reshaped)
#         print(len(result))
        print(result)

        label = np.argmax(result, axis=1)[0]
        print(label)
        text = labels_dict[label] + ": " + str(label)
        
        cv2.rectangle(img, (x, y), (x+w, y+h), colors_dict[label], 2)
        cv2.putText(img, text, (x+2, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, colors_dict[label], 2)
        
    cv2.imshow('MASK_DETECTOR', img)
    key = cv2.waitKey(1)
    
    if key == 27:
        break
cv2.destroyAllWindows()
cap.release()

[[0.00808449 0.9919155 ]]
1
[[0.00225393 0.99774605]]
1
[[0.01219316 0.98780686]]
1
[[0.00166658 0.9983334 ]]
1
[[0.00586092 0.9941391 ]]
1
[[0.0014557  0.99854434]]
1
[[0.03183541 0.9681646 ]]
1
[[0.0103832 0.9896169]]
1
[[0.00111796 0.998882  ]]
1
[[0.81799746 0.18200253]]
0
[[0.00377858 0.9962214 ]]
1
[[0.00126779 0.99873215]]
1
[[0.7619998  0.23800023]]
0
[[0.01112781 0.98887223]]
1
[[0.34930164 0.65069836]]
1
[[0.00608415 0.99391586]]
1
[[0.8551751  0.14482492]]
0
[[0.00598186 0.9940182 ]]
1
[[0.66229266 0.3377073 ]]
0
[[7.4198405e-04 9.9925798e-01]]
1
[[0.0055652 0.9944348]]
1
[[0.5125313 0.4874687]]
0
[[0.00110428 0.99889565]]
1
[[0.03160603 0.9683939 ]]
1
[[0.00672277 0.99327725]]
1
[[0.6929403  0.30705965]]
0
[[0.00187317 0.9981268 ]]
1
[[0.72814983 0.27185017]]
0
[[0.0099378  0.99006224]]
1
[[0.00184098 0.998159  ]]
1
[[0.02514803 0.97485197]]
1
[[0.00218017 0.9978198 ]]
1
[[0.00166569 0.99833435]]
1
[[0.00805084 0.9919492 ]]
1
[[6.4234470e-04 9.9935764e-01]]
1
[[0.0090218  0